In [27]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

In [28]:
input_path = "."

## Export Csv Personendaten

In [29]:
input_file = f"WIAG-Domherren-DB-Lebensdaten_2024-06-04.csv"
input_path_file = os.path.join(input_path, input_file)
persons_df = pd.read_csv(input_path_file, sep=';')
print(len(persons_df))
persons_df.head()

15606


,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia
0,WIAG-Pers-EPISCGatz-03848-001,epc,Ulrich,NaN,NaN,Ulrich,NaN,973,+ 973,"Bischof Augsburg 923-973, Domherr Domstift Aug...",118625284,059-01621-001,Q653546,Q714866,Ulrich_von_Augsburg
1,WIAG-Pers-EPISCGatz-03858-001,epc,Siegfried,NaN,NaN,Siegfried,NaN,1096,+ 1096,"Ernannter Bischof Augsburg 1077-1096, Domherr ...",13805746X,050-06892-001,Q653421,Q64763639,NaN
2,WIAG-Pers-EPISCGatz-04027-001,epc,Norbert,NaN,NaN,Norbert,NaN,NaN,~ 1079/1080,Bischof Chur 1079/1080-1088,NaN,710-00332-001,Q653192,Q35136089,Norbert_von_Hohenwart
3,WIAG-Pers-CANON-91597-001,dreg-can,Sizo,NaN,NaN,Sizo,NaN,NaN,~ um 1099,Domkustos Domstift Augsburg um 1099,NaN,710-00420-001,Q700910,NaN,NaN
4,WIAG-Pers-EPISCGatz-02518-001,epc,Udalschalk,NaN,NaN,Udalschalk,NaN,1202,+ 1202,"Bischof Augsburg 1184-1202, Dompropst Domstift...",138018383,059-01616-001,Q649639,Q68911884,NaN


In [30]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
}
"""
)

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
wiag_ids_df = pd.json_normalize(data['results']['bindings'])

len(wiag_ids_df)

11776

In [31]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [32]:
drop_type_columns(wiag_ids_df)
extract_qid(wiag_ids_df, 'person.value')
wiag_ids_df.columns = ['FactGrid_ID', 'old_wiag_id']
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q16188,WIAG-Pers-EPISCGatz-10475-001
1,Q16194,WIAG-Pers-EPISCGatz-10816-001
2,Q153179,WIAG-Inst-DIOCGatz-001-001
3,Q153216,WIAG-Inst-DIOCGatz-002-001
4,Q153217,WIAG-Inst-DIOCGatz-003-001
...,...,...
11771,Q946227,WIAG-Pers-CANON-53013-001
11772,Q946230,WIAG-Pers-CANON-53022-001
11773,Q946231,WIAG-Pers-CANON-53027-001
11774,Q946232,WIAG-Pers-CANON-53029-001


In [33]:
merged_df = wiag_ids_df.merge(persons_df[['FactGrid_ID', 'id']], on='FactGrid_ID')
merged_df

,FactGrid_ID,old_wiag_id,id
0,Q16188,WIAG-Pers-EPISCGatz-10475-001,WIAG-Pers-EPISCGatz-10475-001
1,Q16194,WIAG-Pers-EPISCGatz-10816-001,WIAG-Pers-EPISCGatz-10816-001
2,Q164286,WIAG-Pers-EPISCGatz-03496-001,WIAG-Pers-EPISCGatz-03496-001
3,Q165581,WIAG-Pers-EPISCGatz-05581-001,WIAG-Pers-EPISCGatz-05581-001
4,Q164124,WIAG-Pers-EPISCGatz-05582-001,WIAG-Pers-EPISCGatz-05582-001
...,...,...,...
7632,Q701496,WIAG-Pers-CANON-15090-001,WIAG-Pers-CANON-15090-001
7633,Q883740,WIAG-Pers-CANON-48925-001,WIAG-Pers-CANON-48925-001
7634,Q883775,WIAG-Pers-CANON-48861-001,WIAG-Pers-CANON-48861-001
7635,Q883788,WIAG-Pers-CANON-15152-001,WIAG-Pers-CANON-15152-001


In [35]:
diff_df = merged_df[merged_df['old_wiag_id'] != merged_df['id']]
diff_df

,FactGrid_ID,old_wiag_id,id
7429,Q883786,WIAG-Pers-CANON-85490-001,WIAG-Pers-CANON-10577-001
7432,Q883793,WIAG-Pers-CANON-85546-001,WIAG-Pers-CANON-15141-001
7438,Q883800,WIAG-Pers-CANON-85584-001,WIAG-Pers-CANON-15162-001
7440,Q883804,WIAG-Pers-CANON-85636-001,WIAG-Pers-CANON-15127-001
7442,Q883806,WIAG-Pers-CANON-86198-001,WIAG-Pers-CANON-15159-001
7457,Q883828,WIAG-Pers-CANON-88498-001,WIAG-Pers-CANON-25794-001
7458,Q883829,WIAG-Pers-CANON-88522-001,WIAG-Pers-CANON-15086-001
7459,Q883830,WIAG-Pers-CANON-88539-001,WIAG-Pers-CANON-26197-001
7460,Q883832,WIAG-Pers-CANON-88566-001,WIAG-Pers-CANON-17772-001
7461,Q883833,WIAG-Pers-CANON-88581-001,WIAG-Pers-CANON-25485-001


In [45]:
fg_qs_csv = diff_df.copy()
fg_qs_csv.columns = ['qid', '-P601', 'P601']
fg_qs_csv['-P601'] = '"' + fg_qs_csv['-P601'] + '"'
fg_qs_csv['P601'] = '"' + fg_qs_csv['P601'] + '"'
fg_qs_csv.set_index('qid')
fg_qs_csv

,qid,-P601,P601
7429,Q883786,"""WIAG-Pers-CANON-85490-001""","""WIAG-Pers-CANON-10577-001"""
7432,Q883793,"""WIAG-Pers-CANON-85546-001""","""WIAG-Pers-CANON-15141-001"""
7438,Q883800,"""WIAG-Pers-CANON-85584-001""","""WIAG-Pers-CANON-15162-001"""
7440,Q883804,"""WIAG-Pers-CANON-85636-001""","""WIAG-Pers-CANON-15127-001"""
7442,Q883806,"""WIAG-Pers-CANON-86198-001""","""WIAG-Pers-CANON-15159-001"""
7457,Q883828,"""WIAG-Pers-CANON-88498-001""","""WIAG-Pers-CANON-25794-001"""
7458,Q883829,"""WIAG-Pers-CANON-88522-001""","""WIAG-Pers-CANON-15086-001"""
7459,Q883830,"""WIAG-Pers-CANON-88539-001""","""WIAG-Pers-CANON-26197-001"""
7460,Q883832,"""WIAG-Pers-CANON-88566-001""","""WIAG-Pers-CANON-17772-001"""
7461,Q883833,"""WIAG-Pers-CANON-88581-001""","""WIAG-Pers-CANON-25485-001"""


In [46]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [48]:
fg_qs_csv.to_csv(f'factgrid_wiag_id_update_{today_string}.csv', index=False)